# MAT281 - Laboratorio N°03



<a id='p1'></a>
## I.- Problema 01


<img src="https://freedesignfile.com/upload/2013/06/Car-logos-1.jpg" width="360" height="360" align="center"/>


El conjunto de datos se denomina `vehiculos_procesado_con_grupos.csv`, el cual contine algunas de las características más importante de un vehículo.

En este ejercicio se tiene como objetivo, es poder clasificar los distintos vehículos basados en las cracterísticas que se presentan a continuación. La dificultad de este ejercicio radíca en que ahora tenemos variables numéricas y variables categóricas.

Lo primero será cargar el conjunto de datos:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans


%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv(os.path.join("data","vehiculos_procesado_con_grupos.csv"), sep=",")\
       .drop(
            ["fabricante", 
             "modelo",
             "transmision", 
             "traccion", 
             "clase", 
             "combustible",
             "consumo"], 
    
          axis=1)

df.head()

En este caso, no solo se tienen datos numéricos, sino que también categóricos. Además, tenemos problemas de datos **vacíos (Nan)**. Así que para resolver este problema, seguiremos varios pasos:

### 1.- Normalizar datos

1.- Cree un conjunto de datos con las variables numéricas, además, para cada dato vacía, rellene con el promedio asociado a esa columna. Finalmente, normalize los datos mediante el procesamiento **MinMaxScaler** de **sklearn**.

2.-  Cree un conjunto de datos con las variables categóricas , además, transforme de variables categoricas a numericas ocupando el comando **get_dummies** de pandas ([referencia](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Explique a grande rasgo como se realiza la codificación de variables numéricas a categóricas.

3.- Junte ambos dataset en uno, llamado **df_procesado**. 


In [ ]:
# 1
# crear dataframe conteninendo solo variables numericas
# notar que convenientemente, las variables no numéricas contienen el string 'tipo'  
df_num = df.drop(df.filter(regex='tipo'), axis=1)
display(df_num.head())
print("Cantidad de datos nulos: ")
df_num.isnull().sum(axis = 0) #chequear cantidad de NaN

In [ ]:
# rellenar celdas vacía con el promedio 
columnas = df_num.columns
[df_num[col].fillna(value = df[col].mean(), inplace = True) for col in columnas]
print("Cantidad de datos nulos: ")
df_num.isnull().sum(axis = 0) #rechequear si hay datos NaN

In [ ]:
# normalizacion usando MinMaxScaler
scaler = MinMaxScaler()
columns = df_num.columns
df_num[columns] = scaler.fit_transform(df[columns])
display(df_num)

In [ ]:
# se inspecciona para ver si funciono mediante scatterplot 2D en dos columnas
ax = sns.scatterplot(data = df_num, x = "desplazamiento", y = "cilindros")

In [ ]:
#2
# crear conjunto con solo variables categóricas
df_cat = df.filter(regex='tipo')
df_cat.head()

In [ ]:
# usar get_dummie para transformar a variable numéricas 
df_cat = pd.get_dummies(df_cat)
df_cat

La función **get_dummies** básicamente toma todas las entradas **únicas** de cada columna de valores categóricos del dataframe original y las convierte en nuevas columnas.  En cada entrada de las filas se va rellenando con un 1 si es que el dato pertenece a la categoría ahora dada por una columna y con 0 en caso contrario.

In [ ]:
# juntar datos númericos y datos dummyficados
df_procesado = pd.concat([df_num,df_cat], axis =1, sort = False).fillna(0)
df_procesado.head()

### 2.- Realizar ajuste mediante kmeans

Una vez depurado el conjunto de datos, es momento de aplicar el algoritmo de **kmeans**.

1. Ajuste el modelo de **kmeans** sobre el conjunto de datos, con un total de **8 clusters**.
2. Asociar a cada individuo el correspondiente cluster y calcular valor de los centroides de cada cluster.
3. Realizar un resumen de las principales cualidades de cada cluster. Para  esto debe calcular (para cluster) las siguientes medidas de resumen:
    * Valor promedio de las variables numérica
    * Moda para las variables numericas
    
    

In [ ]:
# 1
kmeans = KMeans(n_clusters=8)
kmeans.fit(df_procesado)

centroids = kmeans.cluster_centers_ # centros
clusters = kmeans.labels_ #clusters

centroids[1]

In [ ]:
# 2
df_procesado["cluster"] = clusters
df_procesado["cluster"] = df_procesado["cluster"].astype('category')
display(df_procesado.head())
centroids_df = pd.DataFrame(centroids, columns = df_procesado.columns.drop('cluster'))
centroids_df["cluster"] = np.arange(1,9)

In [ ]:
# tanteo en 2D para ver si se alcanzan a divisar los clusters para dos columnas
fig, ax = plt.subplots(figsize=(11,8.5))
sns.scatterplot(
        data = df_procesado,
        x = "consumo_litros_milla",
        y = "co2", 
        hue = "cluster",
        legend = 'full',
        palette ="Set2"
        )

g = sns.scatterplot(
        x = "consumo_litros_milla",
        y = "co2",
        s=100, color = "black", marker = "x",
        data = centroids_df
        )

In [ ]:
# 3
df_aux = df_procesado.drop(df_procesado.filter(regex='_tipo'), axis = 1)
medias = df_aux.groupby('cluster').mean()
moda = df_aux.groupby('cluster').apply(lambda x: x.mode()).drop('cluster', axis = 1)
display(medias)
display(moda)
print("***Recordar que estos datos han sido normalizados según el critero MinMaxScale***")

### 3.- Elegir Número de cluster

Estime mediante la **regla del codo**, el número de cluster apropiados para el caso.
Para efectos prácticos, eliga la siguiente secuencia como número de clusters a comparar:

$$[5, 10, 20, 30, 50, 75, 100, 200, 300]$$

Una ve realizado el gráfico, saque sus propias conclusiones del caso.


In [ ]:
# regla del codo
# advertencia: se requieren moderadamente altos recursos computacionales al correr esta celda (un par de minutos)

Nc = [5,10,20,30,50,75,100,200,300]
kmeans = [KMeans(n_clusters = i) for i in Nc]
score = [kmeans[i].fit(df_procesado).inertia_ for i in range(len(kmeans))]

df_Elbow = pd.DataFrame({'Number of Clusters':Nc,
                        'Score':score})
df_Elbow

In [ ]:
# graficar los datos etiquetados con k-means
fix, ax = plt.subplots(figsize = (11, 8.5))
plt.title('Elbow Curve')
sns.lineplot(x = 'Number of Clusters', y='Score', data=df_Elbow)
g = sns.scatterplot(x = 'Number of Clusters', y='Score', data=df_Elbow)

La mayor diferencia de pendientes se observa entre la pendientes desde 5 a 10 clusters y la de 10 a 20 clusters, por lo que según el criterio visto en clases el número óptimo de clústers sería 10. 

Se podrían haber tomado los años de los vehículos como datos categóticos en vez de numéricos y ver como cambian los clusters.

## II.- Problema 02

<img src="https://live.staticflickr.com/7866/47075467621_85ab810139_c.jpg" align="center"/>

Para el conjunto de datos de **Iris**, se pide realizar una reducción de dimensionalidad ocupando las técnicas de PCA y TSNE (vistas en clases). 

El objetivo es aplicar ambos algoritmos de la siguiente manera:

* Análisis detallado algoritma PCA (tablas, gráficos, etc.)
* Análisis detallado algoritma TSNE (tablas, gráficos, etc.)
* Comparar ambos algoritmos (conclusiones del caso)

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from time import time
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [ ]:
dataset = load_iris()
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
target = 'species'

iris = pd.DataFrame(
    dataset.data,
    columns=features)

iris[target] = dataset.target
iris.head()

In [ ]:
# PCA
scaler = StandardScaler() #reescalado según la normal estándar ie z = (x - u) / s con u, s media y std de la muestra de entrenamiento

#separación de target y datos
X = iris.drop(columns='species') 
y = iris['species']
#obtención de 2 componentes principales vía método PCA
embedding = PCA(n_components=2) 
X_transform = embedding.fit_transform(X)

df_pca = pd.DataFrame(X_transform, columns = ['Score1', 'Score2']) #empacar a dataframe
df_pca['species'] = y

In [ ]:
# Plot Iris PCA

# estilo del grafico de dispersion
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

sns.lmplot(x='Score1',
          y='Score2',
          data = df_pca,  
          fit_reg=False,
          legend=True,
          height=9,
          hue='species',
          scatter_kws = {"s":200, "alpha":0.3} 
          )
plt.title('PCA Results: Iris', weight='bold').set_fontsize('14')
plt.xlabel('PC1', weight='bold').set_fontsize('10')
plt.ylabel('PC2', weight='bold').set_fontsize('10')

Del gráfico de las dos componentes principales desde el método **PCA**  se observa que distingue correctamente la especie 0 del resto, mientras que las otras especies presentan intersección, aunque la mayor parte de los datos son distinguidos correctamente.

In [ ]:
# tSNE
scaler = StandardScaler() #reescalado normal estandar al igual que en PCA
#separar target de datos
X = iris.drop(columns='species')
y = iris['species']
#obtención de 2 componentes principales mediante tSNE    
embedding = TSNE(n_components=2)
X_transform = embedding.fit_transform(X)
    
df_tsne = pd.DataFrame(X_transform,columns = ['_DIM_1_','_DIM_2_'])
df_tsne['species'] = y

In [ ]:
# Plot Digits t-SNE
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

sns.lmplot(x='_DIM_1_',
           y='_DIM_2_',
           data=df_tsne,
           fit_reg=False,
           legend=True,
           height=9,
           hue='species',
           scatter_kws={"s":200, "alpha":0.3})

plt.title('t-SNE Results: Digits', weight='bold').set_fontsize('14')
plt.xlabel('Dimension 1', weight='bold').set_fontsize('10')
plt.ylabel('Dimension 2', weight='bold').set_fontsize('10')

Al igual que en **PCA**, el método **tSNE** distingue correctamente la especie 0. Hay algunos datos clasificados incorrectamente ya que hay una intersección de los clúster para especies 1 y 2, pero en general es bastante efectivo.

Ambos métodos distinguen apropiadamente la especie 0 del resto, sin embargo las especies 1 y 2 no son separadas totalmente. Se realizaron varias iteraciones, **tSNE** siempre agrupa al menos 4 datos en la intersección de las especies 1 y 2 no pudiendo ver claramente una distinción entre este subcojunto de datos. Por otro lado de los gráficos de **PCA**, se puede imaginar una suerte de recta separando las especies pero no totalmente.

En resumen, ambos métodos clasifican con similar efectividad por lo es viable reducir la dimensionalidad del problema, se recomienda **PCA** ya que al correr varias veces los modelos este presenta mayor efectividad al clasificar especies (visualmente) al menos para este conjunto de datos.

In [ ]:
iris.shape